In [198]:
import scipy.io
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import accuracy_score
import torch
import numpy as np

In [199]:
# 分割数据集（无用）
def data_split(data, test_size):
    trn_data = {}
    val_data = {}

    for label, data in data.items():
        lens = len(data[label])
        print(lens)
        trn_data[label] = data[: lens * (1 - test_size)]
        val_data[label] = data[lens * (1 - test_size) :]
    
    return trn_data, val_data

In [200]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [201]:
# 读取mat
trn_mat = scipy.io.loadmat("data/train_data.mat")
trn_data = trn_mat["train"]
# print(len(trn_data))
x_len = len(trn_data)

# 标注数据label
trn_label = [i + 1 for i in range(x_len)]
# print(trn_label)

# 将label与data组合为字典
trn_dict = {}
for i in range(x_len):
    trn_dict[i+1] = trn_data[i]
# print(len(trn_dict))
print(type(trn_dict[1]))

# 读取测试集
val_mat = scipy.io.loadmat('data/test_data.mat')
val_data = val_mat['test']
val_data = val_data
print(type(val_data[1]))
print(len(val_data))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
1000


In [202]:
# 定义SVM
class LinearSVM:
    # 定义学习率、正则化强度、迭代次数
    def __init__(self, lr=1e-3, reg=1e-5, num_iter=1200):
        self.lr = lr
        self.reg = reg
        self.num_iter = num_iter
        
    # 计算损失和梯度
    def compute_loss_and_grad(self, x, y):
        num_trn = x.shape[0] # ==2200
        scores = x.dot(self.W)  # 获取全分数矩阵
        print(scores.shape)
        print(y.shape)
        y -= 1
        
        correct_cls_scores = scores[np.arange(num_trn), y] # 提取正确类别的分数 
        
        margins = np.maximum(0, scores - correct_cls_scores[:, np.newaxis] + 1) # 计算损失边缘
        margins[np.arange(num_trn), y] = 0  # 令正确分类损失为0
        loss = np.sum(margins) / num_trn    # 计算损失
        loss += self.reg * np.sum(self.W * self.W)    # 对损失进行正则化处理，防止过拟合
        
        margins[margins>0] = 1  # 将边缘损失矩阵大于零的部分设为1，方便计算梯度
        margins[np.arange(num_trn), y] = -np.sum(margins, axis=1)   
        dW = x.T.dot(margins) / num_trn # 计算权重矩阵的梯度取平均值，用于更新模型参数
        dW += self.reg * self.W # 正则化梯度，用于惩罚过大的权重
        
        return loss, dW

    # 训练
    def fit(self, x, y):
        num_trn, dim = x.shape
        num_cls = np.max(y)
        self.W = 0.001 * np.random.randn(dim, num_cls)  # 生成一个初始权重矩阵

        for i in range(self.num_iter):
            loss, grad = self.compute_loss_and_grad(x, y)
            self.W -= self.lr * grad    # 梯度下降法更新权重
            if i % 100 == 0:
                print(f"Iteration {i}/{self.num_iter}, Loss: {loss}\ndW: {grad.shape}")
                
    # 预测
    def predict(self, x):
        scores = x.dot(self.W)  # 利用权重进行预测
        if scores.size == 0:
            return []  # 如果输入序列为空，则返回空列表
        return np.argmax(scores, axis=1)    # 输出得分最高的类别

In [203]:
# 收集并转换数据
X = []
Y = []
X_trn = []
X_val = []
Y_trn = []
Y_val = []
for y, x in trn_dict.items():
    X.append(x)
    Y.append([y] * len(x))

for label in range(200):
    xt, xv, yt, yv = train_test_split(X[label], Y[label], test_size=0.25, random_state=42)
    X_trn.append(xt)
    X_val.append(xv)
    Y_trn.append(yt)
    Y_val.append(yv)

X_trn = np.array(X_trn)
Y_trn = np.array(Y_trn)
print(X_trn.shape)

# 重塑数据
x_trn = X_trn.reshape(2200, 28 * 28).astype(np.float64)
x_val = val_data.reshape(val_data.shape[0], -1).astype(np.float64)
y_trn = Y_trn.reshape(2200, )
print(y_trn.shape)
x_trn /= 255.
x_val /= 255.

# # 计算并减去均值图像
# mean_img = np.mean(x_trn, axis=0).astype(np.float64)
# x_trn -= mean_img
# x_val -= mean_img

print(x_trn[0])

(200, 11, 28, 28)
(2200,)
[1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.     

In [204]:
# 训练 SVM
svm = LinearSVM()
svm.fit(x_trn, y_trn)

(2200, 200)
(2200,)
Iteration 0/1200, Loss: 198.9641179966446
dW: (784, 200)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(2200, 200)
(2200,)
(22

IndexError: index -201 is out of bounds for axis 1 with size 200

In [ ]:
# 预测
x_val = np.array(x_val)
print(x_val.shape)
predictions = svm.predict(x_val)

X_val = np.array(X_val)
X_val = X_val.reshape(800, 28 * 28)
print(X_val.shape)
validations = svm.predict(X_val)
acc = accuracy_score(validations, Y_val)

print(len(x_val))
print(f"acc: {acc}")
print(predictions)

(1000, 784)
(800, 784)


ValueError: Found input variables with inconsistent numbers of samples: [800, 200]

In [ ]:
# 将预测结果写入csv
import pandas as pd

submission = pd.read_csv('data/submission.csv', encoding='utf8')
submission['预测结果'] = predictions
submission.to_csv('data/submission.csv', index=False)